# Capstone Project - The Battle of the Neighborhoods (Week 2)

## Table of contents

* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. This report will be targeted to stakeholders interested in opening a restaurant in New York.

Since there are lots of restaurants in NewYork we will try to detect locations that are mostly crowded with restaurants and what category of restaurants are the most. 

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

New York has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Luckily, this dataset exists for free on the web. Here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For convenience, we will use the file placed on on the server, so you we can simply get it. So let's go

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
# import urllib.request
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Next, let's load the data.

In [2]:
data_url = "https://cocl.us/new_york_dataset"
newyork_data = requests.get(data_url).json()

Let's take a quick look at the data.

In [3]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

<a id='item1'></a>

In [4]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [5]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranform the data into a *pandas* dataframe

Creating an empty dataframe.

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

let's loop through the data and fill the dataframe one row at a time.

In [7]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [8]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [10]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


Now we will find out latitude and longitude of each borough by defining a function

In [11]:
def getCordinates(addresses):
    
    boroughs_list = []
    
    for address in addresses:
        geolocator = Nominatim(user_agent="ny_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude


        boroughs_list.append([address,
                         latitude, 
                         longitude])
        
        nearby_boroughs = pd.DataFrame(boroughs_list, columns = ['Borough', 'Latitude', 'Longitude'])
        nearby_boroughs.columns = ['Borough',
                                  'Latitude', 
                                  'Longitude']
    
    return(nearby_boroughs)

Now we will call this function.

In [12]:
boroughs = getCordinates(addresses = neighborhoods['Borough'].unique())
boroughs

,Borough,Latitude,Longitude
0,Bronx,40.846651,-73.878594
1,Manhattan,40.789624,-73.959894
2,Brooklyn,40.650104,-73.949582
3,Queens,40.749824,-73.797634
4,Staten Island,40.583456,-74.149605


#### Now we will create a map of New York with neighborhoods superimposed on top.

In [13]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [14]:
CLIENT_ID = 'KSYAUEOZY3LRZZ3BWTE0AVKPTYEXMFQLYETR2E04KGDAYSSP' # your Foursquare ID
CLIENT_SECRET = 'PAL0SWGAMAQN552DZWPBINZV0Q1VILOW0R5O23DR4ARKOX4M' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentails:
CLIENT_ID: KSYAUEOZY3LRZZ3BWTE0AVKPTYEXMFQLYETR2E04KGDAYSSP
CLIENT_SECRET:PAL0SWGAMAQN552DZWPBINZV0Q1VILOW0R5O23DR4ARKOX4M


#### Lets create a function to explore the neighborhoods

<a id='item2'></a>

In [15]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

def getNearbyVenues(boroughs, names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for borough, name, lat, lng in zip(boroughs, names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough,
            name, 
            lat, 
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


#### Now we will  run the above function on each neighborhood and create a new dataframe called *newyork_venues*.

In [16]:
newyork_venues = getNearbyVenues(boroughs=neighborhoods['Borough'],
                                   names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'])

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [17]:
newyork_venues.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bronx,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Bronx,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
2,Bronx,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Bronx,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Bronx,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station


#### Extracting restaurants from venues and creating a new dataframe called newyork_venues_restrurants.

In [18]:
newyork_venues_restaurant = newyork_venues[newyork_venues['Venue Category'].str.contains('Restaurant')].reset_index(drop=True)

newyork_venues_restaurant.rename(columns={'Venue': 'Restaurant', 'Venue Latitude': 'Restaurant Latitude',  'Venue Longitude': 'Restaurant Longitude', 'Venue Category': 'Restaurant Category'}, inplace=True)
newyork_venues_restaurant

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Restaurant,Restaurant Latitude,Restaurant Longitude,Restaurant Category
0,Bronx,Co-op City,40.874294,-73.829939,Arby's,40.870411,-73.828606,Fast Food Restaurant
1,Bronx,Co-op City,40.874294,-73.829939,Townhouse Restaurant,40.876086,-73.828868,Restaurant
2,Bronx,Eastchester,40.887556,-73.827806,Fish & Ting,40.885656,-73.829197,Caribbean Restaurant
3,Bronx,Eastchester,40.887556,-73.827806,Dyre Fish Market,40.889318,-73.831453,Seafood Restaurant
4,Bronx,Eastchester,40.887556,-73.827806,Royal Caribbean Bakery,40.888249,-73.831661,Caribbean Restaurant
5,Bronx,Eastchester,40.887556,-73.827806,McDonald's,40.885636,-73.826840,Fast Food Restaurant
6,Bronx,Eastchester,40.887556,-73.827806,Xing Lung Chinese Restaurant,40.888785,-73.831226,Chinese Restaurant
7,Bronx,Kingsbridge,40.881687,-73.902818,Leche y Miel,40.883742,-73.901857,Latin American Restaurant
8,Bronx,Kingsbridge,40.881687,-73.902818,El Malecon,40.879338,-73.904457,Caribbean Restaurant
9,Bronx,Kingsbridge,40.881687,-73.902818,El Economico Restaurant,40.879330,-73.904597,Spanish Restaurant


#### Let's check the size of the resulting dataframe

In [19]:
print(newyork_venues_restaurant.shape)

(2502, 8)


#### Lets find out how many Pakistani Reststarunt are in New York

In [20]:
newyork_pakistani_restaurant = newyork_venues_restaurant[newyork_venues_restaurant['Restaurant Category'].str.contains('Pakistani Restaurant')].reset_index(drop=True)
newyork_pakistani_restaurant

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Restaurant,Restaurant Latitude,Restaurant Longitude,Restaurant Category
0,Brooklyn,Downtown,40.690844,-73.983463,BK Jani,40.69105,-73.982912,Pakistani Restaurant


## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Newyork that have most no of restaurant, particularly what category of resturants are the most.

In first step we have collected the required data: location and type (category) of every restaurant in New York . We have also identified that there is only one Pakistani Resturant in New York (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of restaurant across different areas of New York - we will see which areas Newyork that have most no of restaurant, particularly what category of resturants are the most.

In third and final step we will focus on areas and create clusters of locations that meet some basic requirements established in discussion with stakeholders:we will take into consideration locations and present map of all such locations but also create clusters (using k-means clustering) of those locations to identify optimal venue location by stakeholders.



## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data.


#### First let's count the number of restaurants in each neighborhood

In [100]:
newyork_neighborhood_restaurant_count = newyork_venues_restaurant.groupby('Neighborhood').count()
newyork_neighborhood_restaurant_count.head()

,Borough,Neighborhood Latitude,Neighborhood Longitude,Restaurant,Restaurant Latitude,Restaurant Longitude,Restaurant Category
Neighborhood,,,,,,,
Allerton,4,4,4,4,4,4,4
Annadale,1,1,1,1,1,1,1
Arlington,1,1,1,1,1,1,1
Arrochar,5,5,5,5,5,5,5
Arverne,2,2,2,2,2,2,2


#### Now  let's count the number of restaurants in each borough

In [22]:
newyork_borough_restaurant_count = newyork_venues_restaurant.groupby('Borough').count()
newyork_borough_restaurant_count

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Restaurant,Restaurant Latitude,Restaurant Longitude,Restaurant Category
Borough,,,,,,,
Bronx,246,246,246,246,246,246,246
Brooklyn,631,631,631,631,631,631,631
Manhattan,907,907,907,907,907,907,907
Queens,557,557,557,557,557,557,557
Staten Island,161,161,161,161,161,161,161


#### Let's find out how which nighborhooad has the most number of  resturants

In [235]:
newyork_neighborhood_restaurant_count[newyork_neighborhood_restaurant_count.Restaurant == newyork_neighborhood_restaurant_count.Restaurant.max()]

,Borough,Neighborhood Latitude,Neighborhood Longitude,Restaurant,Restaurant Latitude,Restaurant Longitude,Restaurant Category
Neighborhood,,,,,,,
Murray Hill,60,60,60,60,60,60,60


#### Let's find out how which borough has the most number of resturants

In [24]:
newyork_borough_restaurant_count[newyork_borough_restaurant_count.Restaurant == newyork_borough_restaurant_count.Restaurant.max()]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Restaurant,Restaurant Latitude,Restaurant Longitude,Restaurant Category
Borough,,,,,,,
Manhattan,907,907,907,907,907,907,907


#### Let's find out how many unique categories of restaurant can be curated from all the returned venues

In [25]:
print('There are {} unique categories of restaurants.'.format(len(newyork_venues_restaurant['Restaurant Category'].unique())))

There are 88 unique categories of restaurants.


#### Analyze Each Neighborhood

In [26]:
# one hot encoding
newyork_restaurant_neighborhood_onehot = pd.get_dummies(newyork_venues_restaurant[['Restaurant Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newyork_restaurant_neighborhood_onehot['Neighborhood'] = newyork_venues_restaurant['Neighborhood']

# move neighborhood column to the first column
fixed_neighborhood_columns = [newyork_restaurant_neighborhood_onehot.columns[-1]] + list(newyork_restaurant_neighborhood_onehot.columns[:-1])
newyork_restaurant_neighborhood_onehot = newyork_restaurant_neighborhood_onehot[fixed_neighborhood_columns]

newyork_restaurant_neighborhood_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,North Indian Restaurant,Paella Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Polish Restaurant,Ramen Restaurant,Restaurant,Romanian Restaurant,Russian Restaurant,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Soba Restaurant,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
0,Co-op City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Co-op City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Eastchester,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Eastchester,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Eastchester,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Analyze Each Borough

In [99]:
# one hot encoding
newyork_restaurant_borough_onehot = pd.get_dummies(newyork_venues_restaurant[['Restaurant Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newyork_restaurant_borough_onehot['Borough'] = newyork_venues_restaurant['Borough']

# move neighborhood column to the first column
fixed_borough_columns = [newyork_restaurant_borough_onehot.columns[-1]] + list(newyork_restaurant_borough_onehot.columns[:-1])
newyork_restaurant_borough_onehot = newyork_restaurant_borough_onehot[fixed_borough_columns]

newyork_restaurant_borough_onehot.head()

,Borough,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,North Indian Restaurant,Paella Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Polish Restaurant,Ramen Restaurant,Restaurant,Romanian Restaurant,Russian Restaurant,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Soba Restaurant,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
0,Bronx,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bronx,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Bronx,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bronx,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Bronx,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [28]:
newyork_restaurant_neighborhood_onehot.shape

(2502, 89)

In [29]:
newyork_restaurant_borough_onehot.shape

(2502, 89)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [30]:
newyork_restaurant_neighborhood_grouped = newyork_restaurant_neighborhood_onehot.groupby('Neighborhood').mean().reset_index()
newyork_restaurant_neighborhood_grouped.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,North Indian Restaurant,Paella Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Polish Restaurant,Ramen Restaurant,Restaurant,Romanian Restaurant,Russian Restaurant,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Soba Restaurant,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
0,Allerton,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Annadale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Arlington,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Arrochar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Arverne,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Next, let's group rows by borough and by taking the mean of the frequency of occurrence of each category

In [31]:
newyork_restaurant_borough_grouped = newyork_restaurant_borough_onehot.groupby('Borough').mean().reset_index()
newyork_restaurant_borough_grouped

,Borough,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,North Indian Restaurant,Paella Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Polish Restaurant,Ramen Restaurant,Restaurant,Romanian Restaurant,Russian Restaurant,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Soba Restaurant,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
0,Bronx,0.000000,0.008130,0.044715,0.004065,0.000000,0.028455,0.00000,0.000000,0.000000,0.000000,0.000000,0.048780,0.000000,0.130081,0.000000,0.004065,0.000000,0.000000,0.000000,0.000000,0.008130,0.000000,0.000000,0.000000,0.000000,0.117886,0.000000,0.004065,0.000000,0.000000,0.004065,0.000000,0.000000,0.000000,0.000000,0.012195,0.000000,0.000000,0.158537,0.000000,0.008130,0.000000,0.00000,0.000000,0.000000,0.081301,0.000000,0.000000,0.004065,0.109756,0.004065,0.000000,0.000000,0.000000,0.000000,0.004065,0.000000,0.000000,0.008130,0.000000,0.000000,0.024390,0.000000,0.000000,0.000000,0.028455,0.000000,0.000000,0.000000,0.004065,0.000000,0.004065,0.126016,0.000000,0.008130,0.000000,0.000000,0.000000,0.000000,0.000000,0.008130,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004065
1,Brooklyn,0.000000,0.000000,0.066561,0.003170,0.003170,0.015848,0.00000,0.000000,0.000000,0.004754,0.003170,0.055468,0.001585,0.082409,0.000000,0.000000,0.006339,0.000000,0.003170,0.004754,0.004754,0.000000,0.000000,0.001585,0.009509,0.036450,0.001585,0.028526,0.003170,0.000000,0.011094,0.001585,0.001585,0.000000,0.001585,0.025357,0.000000,0.001585,0.104596,0.000000,0.042789,0.000000,0.00317,0.012678,0.000000,0.030111,0.000000,0.000000,0.012678,0.082409,0.026941,0.000000,0.000000,0.019017,0.001585,0.000000,0.001585,0.000000,0.004754,0.004754,0.011094,0.049128,0.000000,0.007924,0.000000,0.028526,0.001585,0.001585,0.000000,0.009509,0.000000,0.011094,0.015848,0.000000,0.044374,0.000000,0.001585,0.001585,0.006339,0.000000,0.039620,0.000000,0.001585,0.006339,0.000000,0.020602,0.000000,0.015848
2,Manhattan,0.001103,0.003308,0.079383,0.003308,0.005513,0.012128,0.00441,0.002205,0.002205,0.001103,0.001103,0.011025,0.002205,0.057332,0.000000,0.000000,0.009923,0.001103,0.003308,0.003308,0.002205,0.000000,0.001103,0.004410,0.009923,0.006615,0.002205,0.049614,0.003308,0.000000,0.019846,0.000000,0.004410,0.001103,0.003308,0.026461,0.000000,0.002205,0.135612,0.003308,0.042999,0.003308,0.00000,0.025358,0.001103,0.017641,0.002205,0.003308,0.034179,0.062845,0.008820,0.001103,0.001103,0.029768,0.001103,0.002205,0.000000,0.002205,0.006615,0.000000,0.013230,0.024256,0.000000,0.000000,0.001103,0.039

#### Let's confirm the new size

In [32]:
newyork_restaurant_borough_grouped.shape

(5, 89)

In [33]:
newyork_restaurant_neighborhood_grouped.shape

(262, 89)

#### Let's print each neighborhood along with the top most common category of restaurant

In [34]:
for hood in newyork_restaurant_neighborhood_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp_n = newyork_restaurant_neighborhood_grouped[newyork_restaurant_neighborhood_grouped['Neighborhood'] == hood].T.reset_index()
    temp_n.columns = ['venue','freq']
    temp_n = temp_n.iloc[1:]
    temp_n['freqa'] = temp_n['freq'].astype(float)
    temp_n = temp_n.round({'freq': 2})
    print(temp_n.sort_values('freq', ascending=False).reset_index(drop=True).head())
    print('\n')

----Allerton----
                     venue  freq  freqa
0       Chinese Restaurant   0.5   0.50
1     Fast Food Restaurant  0.25   0.25
2       Spanish Restaurant  0.25   0.25
3       Persian Restaurant     0   0.00
4  Scandinavian Restaurant     0   0.00


----Annadale----
                     venue freq  freqa
0               Restaurant    1    1.0
1        Afghan Restaurant    0    0.0
2       African Restaurant    0    0.0
3  Scandinavian Restaurant    0    0.0
4       Russian Restaurant    0    0.0


----Arlington----
                     venue freq  freqa
0      American Restaurant    1    1.0
1        Afghan Restaurant    0    0.0
2     Pakistani Restaurant    0    0.0
3  Scandinavian Restaurant    0    0.0
4       Russian Restaurant    0    0.0


----Arrochar----
                       venue freq  freqa
0         Italian Restaurant  0.4    0.4
1   Mediterranean Restaurant  0.2    0.2
2  Middle Eastern Restaurant  0.2    0.2
3          Polish Restaurant  0.2    0.2
4          A

4   Chinese Restaurant  0.0526316  0.052632


----Carroll Gardens----
                 venue       freq     freqa
0   Italian Restaurant   0.458333  0.458333
1      Thai Restaurant  0.0833333  0.083333
2    French Restaurant  0.0833333  0.083333
3  Filipino Restaurant  0.0416667  0.041667
4     Cuban Restaurant  0.0416667  0.041667


----Castleton Corners----
                     venue freq  freqa
0      Japanese Restaurant  0.5    0.5
1               Restaurant  0.5    0.5
2        Afghan Restaurant    0    0.0
3     Pakistani Restaurant    0    0.0
4  Scandinavian Restaurant    0    0.0


----Central Harlem----
                 venue    freq   freqa
0   African Restaurant  0.1875  0.1875
1   Chinese Restaurant   0.125  0.1250
2   Seafood Restaurant   0.125  0.1250
3    French Restaurant   0.125  0.1250
4  American Restaurant   0.125  0.1250


----Charleston----
                     venue freq  freqa
0      American Restaurant  0.5    0.5
1      Japanese Restaurant  0.5    0.5
2      

4       Russian Restaurant         0  0.000000


----Far Rockaway----
                             venue      freq     freqa
0               Chinese Restaurant  0.428571  0.428571
1  Southern / Soul Food Restaurant  0.142857  0.142857
2          Comfort Food Restaurant  0.142857  0.142857
3             Caribbean Restaurant  0.142857  0.142857
4             Fast Food Restaurant  0.142857  0.142857


----Financial District----
                      venue       freq     freqa
0        Mexican Restaurant   0.142857  0.142857
1       American Restaurant   0.142857  0.142857
2        Italian Restaurant   0.142857  0.142857
3       Japanese Restaurant  0.0952381  0.095238
4  Mediterranean Restaurant  0.0952381  0.095238


----Flatbush----
                       venue      freq     freqa
0       Caribbean Restaurant  0.333333  0.333333
1         Mexican Restaurant  0.333333  0.333333
2         Chinese Restaurant  0.166667  0.166667
3  Middle Eastern Restaurant  0.166667  0.166667
4    Scandina

4    Spanish Restaurant  0.0769231  0.076923


----Huguenot----
                     venue freq  freqa
0       Italian Restaurant  0.5    0.5
1  Comfort Food Restaurant  0.5    0.5
2        Afghan Restaurant    0    0.0
3     Pakistani Restaurant    0    0.0
4  Scandinavian Restaurant    0    0.0


----Hunters Point----
                   venue       freq     freqa
0     Italian Restaurant   0.181818  0.181818
1    Japanese Restaurant   0.136364  0.136364
2    American Restaurant  0.0909091  0.090909
3       Asian Restaurant  0.0909091  0.090909
4  Vietnamese Restaurant  0.0454545  0.045455


----Hunts Point----
                  venue      freq     freqa
0    Mexican Restaurant  0.333333  0.333333
1    Spanish Restaurant  0.333333  0.333333
2    Seafood Restaurant  0.333333  0.333333
3     Afghan Restaurant         0  0.000000
4  Pakistani Restaurant         0  0.000000


----Inwood----
                  venue      freq     freqa
0    Mexican Restaurant  0.210526  0.210526
1          

4        Indian Restaurant  0.1    0.1


----Morris Heights----
                 venue freq  freqa
0   Spanish Restaurant    1    1.0
1    Afghan Restaurant    0    0.0
2   Seafood Restaurant    0    0.0
3   Russian Restaurant    0    0.0
4  Romanian Restaurant    0    0.0


----Morris Park----
                  venue      freq     freqa
0      Arepa Restaurant  0.333333  0.333333
1    Spanish Restaurant  0.333333  0.333333
2    Italian Restaurant  0.333333  0.333333
3     Afghan Restaurant         0  0.000000
4  Pakistani Restaurant         0  0.000000


----Morrisania----
                     venue  freq  freqa
0     Fast Food Restaurant   0.5   0.50
1       Seafood Restaurant  0.25   0.25
2       Chinese Restaurant  0.25   0.25
3       Persian Restaurant     0   0.00
4  Scandinavian Restaurant     0   0.00


----Mott Haven----
                       venue  freq  freqa
0         Spanish Restaurant   0.5   0.50
1  Latin American Restaurant  0.25   0.25
2        Peruvian Restaurant  0.

                       venue freq  freqa
0  Latin American Restaurant  0.4    0.4
1         Spanish Restaurant  0.2    0.2
2          Indian Restaurant  0.2    0.2
3       Caribbean Restaurant  0.2    0.2
4          Afghan Restaurant    0    0.0


----Richmond Town----
                     venue freq  freqa
0       Italian Restaurant    1    1.0
1        Afghan Restaurant    0    0.0
2     Pakistani Restaurant    0    0.0
3  Scandinavian Restaurant    0    0.0
4       Russian Restaurant    0    0.0


----Richmond Valley----
                     venue      freq     freqa
0     Fast Food Restaurant  0.666667  0.666667
1       Mexican Restaurant  0.333333  0.333333
2        Afghan Restaurant         0  0.000000
3       Persian Restaurant         0  0.000000
4  Scandinavian Restaurant         0  0.000000


----Ridgewood----
                 venue       freq     freqa
0   Italian Restaurant   0.142857  0.142857
1     Greek Restaurant   0.142857  0.142857
2   Mexican Restaurant  0.0714286  0

                     venue freq  freqa
0     Caribbean Restaurant  0.5    0.5
1       Mexican Restaurant  0.5    0.5
2     Pakistani Restaurant    0    0.0
3  Scandinavian Restaurant    0    0.0
4       Russian Restaurant    0    0.0


----Tottenville----
                     venue      freq     freqa
0       Italian Restaurant  0.666667  0.666667
1       Mexican Restaurant  0.333333  0.333333
2        Afghan Restaurant         0  0.000000
3     Pakistani Restaurant         0  0.000000
4  Scandinavian Restaurant         0  0.000000


----Travis----
                 venue freq  freqa
0   Spanish Restaurant    1    1.0
1    Afghan Restaurant    0    0.0
2   Seafood Restaurant    0    0.0
3   Russian Restaurant    0    0.0
4  Romanian Restaurant    0    0.0


----Tribeca----
                 venue       freq     freqa
0   Italian Restaurant   0.227273  0.227273
1  American Restaurant   0.136364  0.136364
2     Greek Restaurant   0.136364  0.136364
3    French Restaurant  0.0909091  0.0909

#### Let's print each borough along with the top most common category of restaurant

In [36]:
for hood in newyork_restaurant_borough_grouped['Borough']:
    print("----"+hood+"----")
    temp = newyork_restaurant_borough_grouped[newyork_restaurant_borough_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head())
    print('\n')

----Bronx----
                  venue  freq
0    Italian Restaurant  0.16
1    Chinese Restaurant  0.13
2    Spanish Restaurant  0.13
3  Fast Food Restaurant  0.12
4    Mexican Restaurant  0.11


----Brooklyn----
                  venue  freq
0    Italian Restaurant  0.10
1    Mexican Restaurant  0.08
2    Chinese Restaurant  0.08
3   American Restaurant  0.07
4  Caribbean Restaurant  0.06


----Manhattan----
                 venue  freq
0   Italian Restaurant  0.14
1  American Restaurant  0.08
2   Chinese Restaurant  0.06
3   Mexican Restaurant  0.06
4    French Restaurant  0.05


----Queens----
                       venue  freq
0         Chinese Restaurant  0.10
1         Italian Restaurant  0.07
2          Korean Restaurant  0.07
3  Latin American Restaurant  0.05
4         Mexican Restaurant  0.05


----Staten Island----
                 venue  freq
0   Italian Restaurant  0.27
1   Chinese Restaurant  0.10
2           Restaurant  0.08
3     Sushi Restaurant  0.07
4  American Resta

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [103]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most number of Restaurant '.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most number of Restaurant'.format(ind+1))

# create a new dataframe
neighborhoods_restaurant_sorted = pd.DataFrame(columns=columns)
neighborhoods_restaurant_sorted['Neighborhood'] = newyork_restaurant_neighborhood_grouped['Neighborhood']

for ind in np.arange(newyork_restaurant_neighborhood_grouped.shape[0]):
    neighborhoods_restaurant_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_restaurant_neighborhood_grouped.iloc[ind, :], num_top_venues)

neighborhoods_restaurant_sorted

,Neighborhood,1st Most number of Restaurant,2nd Most number of Restaurant,3rd Most number of Restaurant,4th Most number of Restaurant,5th Most number of Restaurant,6th Most number of Restaurant,7th Most number of Restaurant,8th Most number of Restaurant,9th Most number of Restaurant,10th Most number of Restaurant
0,Allerton,Chinese Restaurant,Fast Food Restaurant,Spanish Restaurant,Vietnamese Restaurant,Greek Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
1,Annadale,Restaurant,Vietnamese Restaurant,Kebab Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
2,Arlington,American Restaurant,Vietnamese Restaurant,Hawaiian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant
3,Arrochar,Italian Restaurant,Mediterranean Restaurant,Polish Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant,Greek Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant
4,Arverne,Restaurant,Thai Restaurant,Vietnamese Restaurant,Halal Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
5,Astoria,Middle Eastern Restaurant,Greek Restaurant,Seafood Restaurant,Indian Restaurant,Mediterranean Restaurant,Italian Restaurant,Japanese Restaurant,Comfort Food Restaurant,Latin American Restaurant,Mexican Restaurant
6,Astoria Heights,Italian Restaurant,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
7,Auburndale,Italian Restaurant,Korean Restaurant,Sushi Restaurant,American Restaurant,Fast Food Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
8,Bath Beach,Italian Restaurant,Fast Food Restaurant,Chinese Restaurant,Cantonese Restaurant,Halal Restaurant,Peruvian Restaurant,Dim Sum Restaurant,German Restaurant,Spanish Restaurant,Middle Eastern Restaurant
9,Battery Park City,American Restaurant,Italian Restaurant,Mexican Restaurant,Chinese Restaurant,Sushi Restaurant,Vietnamese Restaurant,Halal Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant


Now let's create the new dataframe and display the top 10 venues for each borough.

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns_b = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns_b.append('{}{} Most number of Restaurant '.format(ind+1, indicators[ind]))
    except:
        columns_b.append('{}th Most number of Restaurant'.format(ind+1))

# create a new dataframe
boroughs_restaurant_sorted = pd.DataFrame(columns=columns_b)
boroughs_restaurant_sorted['Borough'] = newyork_restaurant_borough_grouped['Borough']


for ind in np.arange(newyork_restaurant_borough_grouped.shape[0]):
    boroughs_restaurant_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_restaurant_borough_grouped.iloc[ind, :], num_top_venues)

boroughs_restaurant_sorted

,Borough,1st Most number of Restaurant,2nd Most number of Restaurant,3rd Most number of Restaurant,4th Most number of Restaurant,5th Most number of Restaurant,6th Most number of Restaurant,7th Most number of Restaurant,8th Most number of Restaurant,9th Most number of Restaurant,10th Most number of Restaurant
0,Bronx,Italian Restaurant,Chinese Restaurant,Spanish Restaurant,Fast Food Restaurant,Mexican Restaurant,Latin American Restaurant,Caribbean Restaurant,American Restaurant,Asian Restaurant,Seafood Restaurant
1,Brooklyn,Italian Restaurant,Mexican Restaurant,Chinese Restaurant,American Restaurant,Caribbean Restaurant,Restaurant,Sushi Restaurant,Japanese Restaurant,Thai Restaurant,Fast Food Restaurant
2,Manhattan,Italian Restaurant,American Restaurant,Mexican Restaurant,Chinese Restaurant,Sushi Restaurant,French Restaurant,Japanese Restaurant,Seafood Restaurant,Thai Restaurant,Mediterranean Restaurant
3,Queens,Chinese Restaurant,Korean Restaurant,Italian Restaurant,Mexican Restaurant,Latin American Restaurant,Thai Restaurant,Sushi Restaurant,Japanese Restaurant,Indian Restaurant,Fast Food Restaurant
4,Staten Island,Italian Restaurant,Chinese Restaurant,Restaurant,Mexican Restaurant,Sushi Restaurant,Fast Food Restaurant,American Restaurant,Japanese Restaurant,Spanish Restaurant,Eastern European Restaurant


#### Now we have clear indication of neighborhood with most number of number of restaurants and what category of resturants 

Let us now **cluster** those locations to create **zones containing similar caterory restaurants locations**. Those zones, and their addresses will be the final result of our analysis. 

#### Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 10 clusters.

In [186]:
# set number of clusters
neighborhood_kclusters = 10

newyork_restaurant_neighborhood_grouped_clustering = newyork_restaurant_neighborhood_grouped.drop('Neighborhood', 1)

# run k-means clustering
neighborhood_kmeans = KMeans(n_clusters=neighborhood_kclusters, random_state=0).fit(newyork_restaurant_neighborhood_grouped_clustering)

# check cluster labels generated for each row in the dataframe
neighborhood_kmeans.labels_[0:10] 

array([0, 8, 5, 9, 8, 2, 0, 9, 2, 9])

#### Cluster Borough

Run *k*-means to cluster the borough into 3 clusters.

In [50]:
# set number of clusters
borough_kclusters = 3

newyork_restaurant_borough_grouped_clustering = newyork_restaurant_borough_grouped.drop('Borough', 1)

# run k-means clustering
borough_kmeans = KMeans(n_clusters=borough_kclusters, random_state=0).fit(newyork_restaurant_borough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
borough_kmeans.labels_

array([2, 1, 1, 1, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 restaurant category for each neighborhood.

In [188]:
# add clustering labels
neighborhoods_restaurant_sorted.insert(0, 'Cluster Labels', neighborhood_kmeans.labels_)
newyork_restaurant_neighborhood_merged = neighborhoods


newyork_restaurant_neighborhood_merged = newyork_restaurant_neighborhood_merged.join(neighborhoods_restaurant_sorted.set_index('Neighborhood'), on='Neighborhood')

newyork_restaurant_neighborhood_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most number of Restaurant,2nd Most number of Restaurant,3rd Most number of Restaurant,4th Most number of Restaurant,5th Most number of Restaurant,6th Most number of Restaurant,7th Most number of Restaurant,8th Most number of Restaurant,9th Most number of Restaurant,10th Most number of Restaurant
0,Bronx,Wakefield,40.894705,-73.847201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bronx,Co-op City,40.874294,-73.829939,8.0,Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Kebab Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
2,Bronx,Eastchester,40.887556,-73.827806,3.0,Caribbean Restaurant,Seafood Restaurant,Fast Food Restaurant,Chinese Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
3,Bronx,Fieldston,40.895437,-73.905643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bronx,Riverdale,40.890834,-73.912585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Bronx,Kingsbridge,40.881687,-73.902818,6.0,Mexican Restaurant,Latin American Restaurant,Spanish Restaurant,Chinese Restaurant,Caribbean Restaurant,Seafood Restaurant,Fast Food Restaurant,Restaurant,Hotpot Restaurant,German Restaurant
6,Manhattan,Marble Hill,40.876551,-73.910660,2.0,Seafood Restaurant,Kebab Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
7,Bronx,Woodlawn,40.898273,-73.867315,9.0,American Restaurant,Italian Restaurant,Indian Restaurant,Vietnamese Restaurant,Halal Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
8,Bronx,Norwood,40.877224,-73.879391,6.0,Caribbean Restaurant,Mexican Restaurant,Fast Food Restaurant,Spanish Restaurant,Chinese Restaurant,Restaurant,American Restaurant,Indian Restaurant,Falafel Restaurant,Japanese Curry Restaurant
9,Bronx,Williamsbridge,40.881039,-73.857446,3.0,Caribbean Restaurant,Vietnamese Restaurant,Empanada Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant


Now we will drop the neighborhood that does not have any restaurant.

In [189]:
newyork_restaurant_neighborhood_merged.dropna(inplace=True)
newyork_restaurant_neighborhood_merged['Cluster Labels'] = newyork_restaurant_neighborhood_merged['Cluster Labels'].astype(int)
newyork_restaurant_neighborhood_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most number of Restaurant,2nd Most number of Restaurant,3rd Most number of Restaurant,4th Most number of Restaurant,5th Most number of Restaurant,6th Most number of Restaurant,7th Most number of Restaurant,8th Most number of Restaurant,9th Most number of Restaurant,10th Most number of Restaurant
1,Bronx,Co-op City,40.874294,-73.829939,8,Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Kebab Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
2,Bronx,Eastchester,40.887556,-73.827806,3,Caribbean Restaurant,Seafood Restaurant,Fast Food Restaurant,Chinese Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
5,Bronx,Kingsbridge,40.881687,-73.902818,6,Mexican Restaurant,Latin American Restaurant,Spanish Restaurant,Chinese Restaurant,Caribbean Restaurant,Seafood Restaurant,Fast Food Restaurant,Restaurant,Hotpot Restaurant,German Restaurant
6,Manhattan,Marble Hill,40.876551,-73.910660,2,Seafood Restaurant,Kebab Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
7,Bronx,Woodlawn,40.898273,-73.867315,9,American Restaurant,Italian Restaurant,Indian Restaurant,Vietnamese Restaurant,Halal Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant


Let's create a new dataframe that includes the cluster as well as the top 10 restaurant category for each borough.

In [109]:
# add clustering labels
boroughs_restaurant_sorted.insert(0, 'Cluster Labels', borough_kmeans.labels_)
newyork_restaurant_borough_merged = boroughs


newyork_restaurant_borough_merged = newyork_restaurant_borough_merged.join(boroughs_restaurant_sorted.set_index('Borough'), on='Borough')


newyork_restaurant_borough_merged

,Borough,Latitude,Longitude,Cluster Labels,1st Most number of Restaurant,2nd Most number of Restaurant,3rd Most number of Restaurant,4th Most number of Restaurant,5th Most number of Restaurant,6th Most number of Restaurant,7th Most number of Restaurant,8th Most number of Restaurant,9th Most number of Restaurant,10th Most number of Restaurant
0,Bronx,40.846651,-73.878594,2,Italian Restaurant,Chinese Restaurant,Spanish Restaurant,Fast Food Restaurant,Mexican Restaurant,Latin American Restaurant,Caribbean Restaurant,American Restaurant,Asian Restaurant,Seafood Restaurant
1,Manhattan,40.789624,-73.959894,1,Italian Restaurant,American Restaurant,Mexican Restaurant,Chinese Restaurant,Sushi Restaurant,French Restaurant,Japanese Restaurant,Seafood Restaurant,Thai Restaurant,Mediterranean Restaurant
2,Brooklyn,40.650104,-73.949582,1,Italian Restaurant,Mexican Restaurant,Chinese Restaurant,American Restaurant,Caribbean Restaurant,Restaurant,Sushi Restaurant,Japanese Restaurant,Thai Restaurant,Fast Food Restaurant
3,Queens,40.749824,-73.797634,1,Chinese Restaurant,Korean Restaurant,Italian Restaurant,Mexican Restaurant,Latin American Restaurant,Thai Restaurant,Sushi Restaurant,Japanese Restaurant,Indian Restaurant,Fast Food Restaurant
4,Staten Island,40.583456,-74.149605,0,Italian Restaurant,Chinese Restaurant,Restaurant,Mexican Restaurant,Sushi Restaurant,Fast Food Restaurant,American Restaurant,Japanese Restaurant,Spanish Restaurant,Eastern European Restaurant


Finally, let's visualize the resulting clusters

In [190]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(neighborhood_kclusters)
ys = [i + x + (i*x)**2 for i in range(neighborhood_kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newyork_restaurant_neighborhood_merged['Latitude'], newyork_restaurant_neighborhood_merged['Longitude'], newyork_restaurant_neighborhood_merged['Neighborhood'], newyork_restaurant_neighborhood_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now borough

In [72]:
# create map
map_borough_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(borough_kclusters)
ys = [i + x + (i*x)**2 for i in range(borough_kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newyork_restaurant_borough_merged['Latitude'], newyork_restaurant_borough_merged['Longitude'], newyork_restaurant_borough_merged['Borough'], newyork_restaurant_borough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_borough_clusters)
       
map_borough_clusters

#### Now, we examine each cluster and determine the discriminating restaurant categories that distinguish each cluster.

#### Cluster 1

In [191]:
newyork_restaurant_neighborhood_merged.loc[newyork_restaurant_neighborhood_merged['Cluster Labels'] == 0, newyork_restaurant_neighborhood_merged.columns[[0]+[1] + list(range(5, newyork_restaurant_neighborhood_merged.shape[1]))]]

,Borough,Neighborhood,1st Most number of Restaurant,2nd Most number of Restaurant,3rd Most number of Restaurant,4th Most number of Restaurant,5th Most number of Restaurant,6th Most number of Restaurant,7th Most number of Restaurant,8th Most number of Restaurant,9th Most number of Restaurant,10th Most number of Restaurant
13,Bronx,Bedford Park,Chinese Restaurant,Mexican Restaurant,Italian Restaurant,Spanish Restaurant,Vietnamese Restaurant,Greek Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant
18,Bronx,West Farms,Latin American Restaurant,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
26,Bronx,Soundview,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Halal Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
29,Bronx,Country Club,Chinese Restaurant,Vietnamese Restaurant,Empanada Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant
45,Bronx,Edenwald,Chinese Restaurant,Vietnamese Restaurant,Empanada Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant
50,Brooklyn,Gravesend,Italian Restaurant,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
53,Brooklyn,Manhattan Terrace,Japanese Restaurant,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
60,Brooklyn,Brownsville,Restaurant,Spanish Restaurant,Chinese Restaurant,Vietnamese Restaurant,Greek Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
83,Brooklyn,Marine Park,Chinese Restaurant,Vietnamese Restaurant,Empanada Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant
136,Queens,Kew Gardens,Chinese Restaurant,Indian Restaurant,New American Restaurant,American Restaurant,Italian Restaurant,South American Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant


#### Cluster 2

In [192]:
newyork_restaurant_neighborhood_merged.loc[newyork_restaurant_neighborhood_merged['Cluster Labels'] == 1, newyork_restaurant_neighborhood_merged.columns[[0]+[1] + list(range(5, newyork_restaurant_neighborhood_merged.shape[1]))]]

,Borough,Neighborhood,1st Most number of Restaurant,2nd Most number of Restaurant,3rd Most number of Restaurant,4th Most number of Restaurant,5th Most number of Restaurant,6th Most number of Restaurant,7th Most number of Restaurant,8th Most number of Restaurant,9th Most number of Restaurant,10th Most number of Restaurant
14,Bronx,University Heights,African Restaurant,Latin American Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Hawaiian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
17,Bronx,East Tremont,Fast Food Restaurant,Paella Restaurant,Vietnamese Restaurant,Kebab Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
23,Bronx,Longwood,Latin American Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Halal Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
25,Bronx,Morrisania,Fast Food Restaurant,Seafood Restaurant,Chinese Restaurant,Egyptian Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
31,Bronx,Westchester Square,Fast Food Restaurant,Latin American Restaurant,Asian Restaurant,Mexican Restaurant,Spanish Restaurant,Vietnamese Restaurant,Halal Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant
72,Brooklyn,East New York,Fast Food Restaurant,Caribbean Restaurant,Spanish Restaurant,Chinese Restaurant,Vietnamese Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
80,Brooklyn,Borough Park,Fast Food Restaurant,American Restaurant,Restaurant,Chinese Restaurant,Vietnamese Restaurant,Halal Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
148,Queens,South Ozone Park,Fast Food Restaurant,Vietnamese Restaurant,Venezuelan Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant
166,Queens,Rochdale,Southern / Soul Food Restaurant,Fast Food Restaurant,Chinese Restaurant,Vietnamese Restaurant,Greek Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
167,Queens,Springfield Gardens,Fast Food Restaurant,Chinese Restaurant,Vietnamese Restaurant,Empanada Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant


#### Cluster 3

In [193]:
newyork_restaurant_neighborhood_merged.loc[newyork_restaurant_neighborhood_merged['Cluster Labels'] == 2, newyork_restaurant_neighborhood_merged.columns[[0]+[1] + list(range(5, newyork_restaurant_neighborhood_merged.shape[1]))]]

,Borough,Neighborhood,1st Most number of Restaurant,2nd Most number of Restaurant,3rd Most number of Restaurant,4th Most number of Restaurant,5th Most number of Restaurant,6th Most number of Restaurant,7th Most number of Restaurant,8th Most number of Restaurant,9th Most number of Restaurant,10th Most number of Restaurant
6,Manhattan,Marble Hill,Seafood Restaurant,Kebab Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
12,Bronx,City Island,Seafood Restaurant,American Restaurant,Italian Restaurant,French Restaurant,Spanish Restaurant,Greek Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant
22,Bronx,Port Morris,Latin American Restaurant,Restaurant,Peruvian Restaurant,Spanish Restaurant,Vietnamese Restaurant,Greek Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant
27,Bronx,Clason Point,South American Restaurant,Vietnamese Restaurant,Egyptian Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
32,Bronx,Van Nest,Middle Eastern Restaurant,Vietnamese Restaurant,Halal Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
35,Bronx,Spuyten Duyvil,Asian Restaurant,Thai Restaurant,Vietnamese Restaurant,Halal Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
44,Bronx,Unionport,Latin American Restaurant,Seafood Restaurant,Indian Restaurant,Fast Food Restaurant,Comfort Food Restaurant,Spanish Restaurant,Caribbean Restaurant,Indonesian Restaurant,German Restaurant,Ethiopian Restaurant
49,Brooklyn,Greenpoint,Mexican Restaurant,French Restaurant,Polish Restaurant,Restaurant,Sushi Restaurant,Vietnamese Restaurant,Szechuan Restaurant,Indian Restaurant,New American Restaurant,Falafel Restaurant
51,Brooklyn,Brighton Beach,Eastern European Restaurant,Restaurant,Russian Restaurant,Sushi Restaurant,Korean Restaurant,Fast Food Restaurant,Mediterranean Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant
57,Brooklyn,Kensington,Thai Restaurant,Restaurant,Japanese Restaurant,Mexican Restaurant,Greek Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant


#### Cluster 4

In [194]:
newyork_restaurant_neighborhood_merged.loc[newyork_restaurant_neighborhood_merged['Cluster Labels'] == 3, newyork_restaurant_neighborhood_merged.columns[[0]+[1] + list(range(5, newyork_restaurant_neighborhood_merged.shape[1]))]]

,Borough,Neighborhood,1st Most number of Restaurant,2nd Most number of Restaurant,3rd Most number of Restaurant,4th Most number of Restaurant,5th Most number of Restaurant,6th Most number of Restaurant,7th Most number of Restaurant,8th Most number of Restaurant,9th Most number of Restaurant,10th Most number of Restaurant
2,Bronx,Eastchester,Caribbean Restaurant,Seafood Restaurant,Fast Food Restaurant,Chinese Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
9,Bronx,Williamsbridge,Caribbean Restaurant,Vietnamese Restaurant,Empanada Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant
41,Bronx,Olinville,Caribbean Restaurant,Vietnamese Restaurant,Empanada Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant
56,Brooklyn,East Flatbush,Caribbean Restaurant,Fast Food Restaurant,Chinese Restaurant,Vietnamese Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant
74,Brooklyn,Canarsie,Caribbean Restaurant,Asian Restaurant,Chinese Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant
75,Brooklyn,Flatlands,Fast Food Restaurant,Caribbean Restaurant,Vietnamese Restaurant,Empanada Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant
78,Brooklyn,Coney Island,Vegetarian / Vegan Restaurant,Caribbean Restaurant,Vietnamese Restaurant,Halal Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
164,Queens,South Jamaica,Vegetarian / Vegan Restaurant,Caribbean Restaurant,Vietnamese Restaurant,Halal Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
165,Queens,St. Albans,Caribbean Restaurant,Fast Food Restaurant,Southern / Soul Food Restaurant,Chinese Restaurant,Vietnamese Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
168,Queens,Cambria Heights,Caribbean Restaurant,Restaurant,Chinese Restaurant,Vietnamese Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant


#### Cluster 5

In [195]:
newyork_restaurant_neighborhood_merged.loc[newyork_restaurant_neighborhood_merged['Cluster Labels'] == 4, newyork_restaurant_neighborhood_merged.columns[[0]+[1] + list(range(5, newyork_restaurant_neighborhood_merged.shape[1]))]]

,Borough,Neighborhood,1st Most number of Restaurant,2nd Most number of Restaurant,3rd Most number of Restaurant,4th Most number of Restaurant,5th Most number of Restaurant,6th Most number of Restaurant,7th Most number of Restaurant,8th Most number of Restaurant,9th Most number of Restaurant,10th Most number of Restaurant
34,Bronx,Belmont,Italian Restaurant,Spanish Restaurant,Mexican Restaurant,Eastern European Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Southern / Soul Food Restaurant,Japanese Restaurant,American Restaurant,Austrian Restaurant
39,Bronx,Edgewater Park,Italian Restaurant,American Restaurant,Japanese Restaurant,Asian Restaurant,Vietnamese Restaurant,Halal Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant
206,Staten Island,Mariner's Harbor,Italian Restaurant,Vietnamese Restaurant,Halal Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
217,Staten Island,Tottenville,Italian Restaurant,Mexican Restaurant,Vietnamese Restaurant,Halal Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
230,Staten Island,Old Town,Italian Restaurant,American Restaurant,Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant,Halal Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant
234,Staten Island,New Dorp Beach,Italian Restaurant,Restaurant,Vietnamese Restaurant,Greek Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
251,Staten Island,Richmond Town,Italian Restaurant,Vietnamese Restaurant,Halal Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
252,Staten Island,Shore Acres,Italian Restaurant,Restaurant,Chinese Restaurant,Vietnamese Restaurant,Greek Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
287,Staten Island,Egbertville,Italian Restaurant,Vietnamese Restaurant,Halal Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
292,Staten Island,Lighthouse Hill,Italian Restaurant,Vietnamese Restaurant,Halal Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant


#### Cluster 6

In [196]:
newyork_restaurant_neighborhood_merged.loc[newyork_restaurant_neighborhood_merged['Cluster Labels'] == 5, newyork_restaurant_neighborhood_merged.columns[[0]+[1] + list(range(5, newyork_restaurant_neighborhood_merged.shape[1]))]]

,Borough,Neighborhood,1st Most number of Restaurant,2nd Most number of Restaurant,3rd Most number of Restaurant,4th Most number of Restaurant,5th Most number of Restaurant,6th Most number of Restaurant,7th Most number of Restaurant,8th Most number of Restaurant,9th Most number of Restaurant,10th Most number of Restaurant
73,Brooklyn,Starrett City,Caribbean Restaurant,American Restaurant,Chinese Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant
85,Brooklyn,Sea Gate,American Restaurant,Vietnamese Restaurant,Hawaiian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant
219,Staten Island,Silver Lake,American Restaurant,Vietnamese Restaurant,Hawaiian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant
227,Staten Island,Arlington,American Restaurant,Vietnamese Restaurant,Hawaiian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant
239,Staten Island,Charleston,American Restaurant,Japanese Restaurant,Vietnamese Restaurant,Hawaiian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
240,Staten Island,Rossville,American Restaurant,Chinese Restaurant,Vietnamese Restaurant,Hawaiian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
280,Brooklyn,Vinegar Hill,Latin American Restaurant,American Restaurant,Vietnamese Restaurant,Hawaiian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
281,Brooklyn,Weeksville,American Restaurant,Chinese Restaurant,Vietnamese Restaurant,Hawaiian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
284,Staten Island,Manor Heights,American Restaurant,Chinese Restaurant,Sushi Restaurant,Vietnamese Restaurant,Halal Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant


#### Cluster 7

In [197]:
newyork_restaurant_neighborhood_merged.loc[newyork_restaurant_neighborhood_merged['Cluster Labels'] == 6, newyork_restaurant_neighborhood_merged.columns[[0]+[1] + list(range(5, newyork_restaurant_neighborhood_merged.shape[1]))]]

,Borough,Neighborhood,1st Most number of Restaurant,2nd Most number of Restaurant,3rd Most number of Restaurant,4th Most number of Restaurant,5th Most number of Restaurant,6th Most number of Restaurant,7th Most number of Restaurant,8th Most number of Restaurant,9th Most number of Restaurant,10th Most number of Restaurant
5,Bronx,Kingsbridge,Mexican Restaurant,Latin American Restaurant,Spanish Restaurant,Chinese Restaurant,Caribbean Restaurant,Seafood Restaurant,Fast Food Restaurant,Restaurant,Hotpot Restaurant,German Restaurant
8,Bronx,Norwood,Caribbean Restaurant,Mexican Restaurant,Fast Food Restaurant,Spanish Restaurant,Chinese Restaurant,Restaurant,American Restaurant,Indian Restaurant,Falafel Restaurant,Japanese Curry Restaurant
10,Bronx,Baychester,Fast Food Restaurant,Mexican Restaurant,Spanish Restaurant,Vietnamese Restaurant,Halal Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
16,Bronx,Fordham,Fast Food Restaurant,Spanish Restaurant,Latin American Restaurant,Chinese Restaurant,Caribbean Restaurant,African Restaurant,American Restaurant,Mexican Restaurant,Greek Restaurant,Restaurant
19,Bronx,High Bridge,Chinese Restaurant,Latin American Restaurant,Asian Restaurant,Mexican Restaurant,Spanish Restaurant,Vietnamese Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant
20,Bronx,Melrose,Mexican Restaurant,Vietnamese Restaurant,Halal Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
24,Bronx,Hunts Point,Seafood Restaurant,Mexican Restaurant,Spanish Restaurant,Greek Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant
30,Bronx,Parkchester,American Restaurant,Caribbean Restaurant,Asian Restaurant,Chinese Restaurant,Mexican Restaurant,Restaurant,Italian Restaurant,Israeli Restaurant,Fast Food Restaurant,Filipino Restaurant
38,Bronx,Schuylerville,Mexican Restaurant,American Restaurant,Fast Food Restaurant,Italian Restaurant,Vietnamese Restaurant,Halal Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
43,Bronx,Concourse,Caribbean Restaurant,Fast Food Restaurant,Italian Restaurant,Mexican Restaurant,Spanish Restaurant,Chinese Restaurant,Halal Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant


#### Cluster 8

In [177]:
newyork_restaurant_neighborhood_merged.loc[newyork_restaurant_neighborhood_merged['Cluster Labels'] == 7, newyork_restaurant_neighborhood_merged.columns[[0]+[1] + list(range(5, newyork_restaurant_neighborhood_merged.shape[1]))]]

,Borough,Neighborhood,1st Most number of Restaurant,2nd Most number of Restaurant,3rd Most number of Restaurant,4th Most number of Restaurant,5th Most number of Restaurant,6th Most number of Restaurant,7th Most number of Restaurant,8th Most number of Restaurant,9th Most number of Restaurant,10th Most number of Restaurant
9,Bronx,Williamsbridge,Caribbean Restaurant,Vietnamese Restaurant,Empanada Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant
41,Bronx,Olinville,Caribbean Restaurant,Vietnamese Restaurant,Empanada Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant
169,Queens,Rosedale,Caribbean Restaurant,Vietnamese Restaurant,Empanada Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant
188,Queens,Laurelton,Caribbean Restaurant,Vietnamese Restaurant,Empanada Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant
282,Brooklyn,Broadway Junction,Caribbean Restaurant,Vietnamese Restaurant,Empanada Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant


#### Cluster 9

In [198]:
newyork_restaurant_neighborhood_merged.loc[newyork_restaurant_neighborhood_merged['Cluster Labels'] == 8, newyork_restaurant_neighborhood_merged.columns[[0]+[1] + list(range(5, newyork_restaurant_neighborhood_merged.shape[1]))]]

,Borough,Neighborhood,1st Most number of Restaurant,2nd Most number of Restaurant,3rd Most number of Restaurant,4th Most number of Restaurant,5th Most number of Restaurant,6th Most number of Restaurant,7th Most number of Restaurant,8th Most number of Restaurant,9th Most number of Restaurant,10th Most number of Restaurant
1,Bronx,Co-op City,Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Kebab Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
82,Brooklyn,Gerritsen Beach,Seafood Restaurant,Restaurant,Egyptian Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
157,Queens,Kew Gardens Hills,Restaurant,Sushi Restaurant,Vietnamese Restaurant,Greek Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
177,Queens,Arverne,Restaurant,Thai Restaurant,Vietnamese Restaurant,Halal Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
189,Queens,Lefrak City,Restaurant,Mexican Restaurant,Vietnamese Restaurant,Halal Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
208,Staten Island,Castleton Corners,Japanese Restaurant,Restaurant,Vietnamese Restaurant,Greek Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
215,Staten Island,Annadale,Restaurant,Vietnamese Restaurant,Kebab Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
243,Staten Island,Heartland Village,Restaurant,Vietnamese Restaurant,Kebab Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant
278,Queens,Blissville,Restaurant,Vietnamese Restaurant,Kebab Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant


#### Cluster 10

In [200]:
newyork_restaurant_neighborhood_merged.loc[newyork_restaurant_neighborhood_merged['Cluster Labels'] == 9, newyork_restaurant_neighborhood_merged.columns[[0]+[1] + list(range(5, newyork_restaurant_neighborhood_merged.shape[1]))]]

,Borough,Neighborhood,1st Most number of Restaurant,2nd Most number of Restaurant,3rd Most number of Restaurant,4th Most number of Restaurant,5th Most number of Restaurant,6th Most number of Restaurant,7th Most number of Restaurant,8th Most number of Restaurant,9th Most number of Restaurant,10th Most number of Restaurant
7,Bronx,Woodlawn,American Restaurant,Italian Restaurant,Indian Restaurant,Vietnamese Restaurant,Halal Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
11,Bronx,Pelham Parkway,Italian Restaurant,Chinese Restaurant,Sushi Restaurant,Vietnamese Restaurant,Greek Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant
28,Bronx,Throgs Neck,American Restaurant,Asian Restaurant,Italian Restaurant,Vietnamese Restaurant,Hawaiian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
33,Bronx,Morris Park,Arepa Restaurant,Italian Restaurant,Spanish Restaurant,Vietnamese Restaurant,Greek Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant
36,Bronx,North Riverdale,Italian Restaurant,Chinese Restaurant,Sushi Restaurant,Mexican Restaurant,Vietnamese Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
37,Bronx,Pelham Bay,Italian Restaurant,Fast Food Restaurant,Chinese Restaurant,Latin American Restaurant,Asian Restaurant,Thai Restaurant,Mexican Restaurant,Vietnamese Restaurant,Falafel Restaurant,Filipino Restaurant
46,Brooklyn,Bay Ridge,Italian Restaurant,American Restaurant,Greek Restaurant,Chinese Restaurant,Thai Restaurant,Sushi Restaurant,Caucasian Restaurant,Indian Restaurant,Mediterranean Restaurant,Mexican Restaurant
47,Brooklyn,Bensonhurst,Sushi Restaurant,Italian Restaurant,Chinese Restaurant,Shabu-Shabu Restaurant,Hotpot Restaurant,Vietnamese Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant
52,Brooklyn,Sheepshead Bay,Turkish Restaurant,Restaurant,American Restaurant,Italian Restaurant,Russian Restaurant,Vietnamese Restaurant,Gluten-free Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant
58,Brooklyn,Windsor Terrace,American Restaurant,Italian Restaurant,Middle Eastern Restaurant,Chinese Restaurant,Sushi Restaurant,Vietnamese Restaurant,Halal Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant


#### Now, we examine each cluster of borough and determine the discriminating restaurant categories that distinguish each cluster.

#### Cluster 1

In [90]:
newyork_restaurant_borough_merged.loc[newyork_restaurant_borough_merged['Cluster Labels'] == 0, newyork_restaurant_borough_merged.columns[[0]+[4] + list(range(5, newyork_restaurant_borough_merged.shape[1]))]]

,Borough,1st Most number of Restaurant,2nd Most number of Restaurant,3rd Most number of Restaurant,4th Most number of Restaurant,5th Most number of Restaurant,6th Most number of Restaurant,7th Most number of Restaurant,8th Most number of Restaurant,9th Most number of Restaurant,10th Most number of Restaurant
4,Staten Island,Italian Restaurant,Chinese Restaurant,Restaurant,Mexican Restaurant,Sushi Restaurant,Fast Food Restaurant,American Restaurant,Japanese Restaurant,Spanish Restaurant,Eastern European Restaurant


#### Cluster 2

In [94]:
newyork_restaurant_borough_merged.loc[newyork_restaurant_borough_merged['Cluster Labels'] == 1, newyork_restaurant_borough_merged.columns[[0]+[4] + list(range(5, newyork_restaurant_borough_merged.shape[1]))]]

,Borough,1st Most number of Restaurant,2nd Most number of Restaurant,3rd Most number of Restaurant,4th Most number of Restaurant,5th Most number of Restaurant,6th Most number of Restaurant,7th Most number of Restaurant,8th Most number of Restaurant,9th Most number of Restaurant,10th Most number of Restaurant
1,Manhattan,Italian Restaurant,American Restaurant,Mexican Restaurant,Chinese Restaurant,Sushi Restaurant,French Restaurant,Japanese Restaurant,Seafood Restaurant,Thai Restaurant,Mediterranean Restaurant
2,Brooklyn,Italian Restaurant,Mexican Restaurant,Chinese Restaurant,American Restaurant,Caribbean Restaurant,Restaurant,Sushi Restaurant,Japanese Restaurant,Thai Restaurant,Fast Food Restaurant
3,Queens,Chinese Restaurant,Korean Restaurant,Italian Restaurant,Mexican Restaurant,Latin American Restaurant,Thai Restaurant,Sushi Restaurant,Japanese Restaurant,Indian Restaurant,Fast Food Restaurant


#### Cluster 3

In [95]:
newyork_restaurant_borough_merged.loc[newyork_restaurant_borough_merged['Cluster Labels'] == 2, newyork_restaurant_borough_merged.columns[[0]+[4] + list(range(5, newyork_restaurant_borough_merged.shape[1]))]]

,Borough,1st Most number of Restaurant,2nd Most number of Restaurant,3rd Most number of Restaurant,4th Most number of Restaurant,5th Most number of Restaurant,6th Most number of Restaurant,7th Most number of Restaurant,8th Most number of Restaurant,9th Most number of Restaurant,10th Most number of Restaurant
0,Bronx,Italian Restaurant,Chinese Restaurant,Spanish Restaurant,Fast Food Restaurant,Mexican Restaurant,Latin American Restaurant,Caribbean Restaurant,American Restaurant,Asian Restaurant,Seafood Restaurant


## Results and Discussion <a name="results"></a>

Our analysis shows that there is a great number of restaurants in New York, but there is only one Pakistani Restaurant in Downtown, Brooklyn. There are 88 unique categoryies of restaurant. Most number of resturant are in Manhattan and in neighborhoods most number of restaurants are in Murray Hill 

After directing our attention we first created a dense grid of location; then we identify areas as wich type of restaurant are there most. These locations were then clustered to create zones of interest which contain greatest number of similar locations.

Result of all these zones containing category of most numbers of resturants. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! Purpose of this analysis was to only provide info on areas with most number of restaurants in New York - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to explore New York areas with most number of resturant and areas with most number of restaurants (particularly Pakistani restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new  restaurant. By collecting resturant data from Foursquare data we have first classified neighborhood and then boroughs and then generated extensive collection of locations which satisfy our requirements. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) were created to be used as starting points for final exploration by stakeholders.

Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.